## Step 1: Mount Google Drive (for Checkpoints)

In [ ]:
from google.colab import drive
import os

# Mount Drive
drive.mount('/content/drive')

# Create checkpoint directory (SEPARATE from LibriSpeech clean)
checkpoint_dir = '/content/drive/MyDrive/librispeech_other_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)
print(f"✅ Checkpoints will be saved to: {checkpoint_dir}")
print(f"   (Isolated from LibriSpeech 'clean' checkpoints)")
print(f"   Testing on 'other' subset (more challenging audio)")

## Step 2: Install Dependencies

In [ ]:
%%capture
# Install required packages
!pip install -q datasets[audio] torch torchaudio transformers accelerate librosa soundfile

# Note: datasets[audio] includes audio decoding dependencies

In [ ]:
# Verify GPU
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

## Step 3: Clone Model Code from GitHub

In [ ]:
import os

# Check if already cloned
repo_root = '/content/VoiceFlow-Intelligence-Platform'
if os.path.exists(repo_root):
    print("✅ Repository already exists, pulling latest changes...")
    !cd /content/VoiceFlow-Intelligence-Platform && git pull origin main
else:
    print("Cloning repository...")
    %cd /content
    !git clone https://github.com/FCHEHIDI/VoiceFlow-Intelligence-Platform.git
    
# Verify the structure and find the correct path
print("\n🔍 Detecting repository structure...")

# Colab sometimes creates deeply nested structures. Check all possibilities:
possible_paths = [
    '/content/VoiceFlow-Intelligence-Platform/voiceflow-ml/VoiceFlow-Intelligence-Platform/voiceflow-ml',
    '/content/VoiceFlow-Intelligence-Platform/VoiceFlow-Intelligence-Platform/voiceflow-ml',
    '/content/VoiceFlow-Intelligence-Platform/voiceflow-ml',
]

voiceflow_ml_path = None
for path in possible_paths:
    models_path = os.path.join(path, 'models')
    if os.path.exists(path) and os.path.exists(models_path):
        voiceflow_ml_path = path
        print(f"✅ Found voiceflow-ml with models/ at: {path}")
        break

if not voiceflow_ml_path:
    print(f"\n❌ Could not find voiceflow-ml with models/ directory")
    print(f"\n📂 /content/VoiceFlow-Intelligence-Platform/:")
    !ls -la /content/VoiceFlow-Intelligence-Platform/
    raise FileNotFoundError("Could not find voiceflow-ml/models/")

# Verify critical files exist
models_path = os.path.join(voiceflow_ml_path, 'models')
diarization_path = os.path.join(models_path, 'diarization')
model_file = os.path.join(diarization_path, 'model.py')

print(f"✅ models/diarization/model.py: {'exists' if os.path.exists(model_file) else 'MISSING'}")

if not os.path.exists(model_file):
    raise FileNotFoundError(f"model.py not found at {model_file}")

# Store the path for next cells
%store voiceflow_ml_path
print(f"\n✅ Repository verified successfully!")
print(f"📁 Using: {voiceflow_ml_path}")

## Step 4: Load VoxConverse (Real Diarization Dataset!)

In [ ]:
from datasets import load_dataset, Audio

print("Loading VoxConverse in STREAMING mode...")
print("⚠️ Real diarization annotations from multi-speaker conversations!\n")

# VoxConverse: Multi-speaker conversations with real diarization labels
# Alternative to AMI (which uses deprecated dataset scripts)
DATASET_NAME = "diarizers-community/callhome"

# Load train split (streaming)
# Fallback: Use LibriSpeech with different subset for more speakers
print("🔄 Using LibriSpeech 'other' subset (more diverse speakers)...")
train_dataset = load_dataset(
    "librispeech_asr",
    "other",  # 'other' subset: 496 hours, more speakers
    split="train.500",
    streaming=True,
    trust_remote_code=True
)

# Load validation split
val_dataset = load_dataset(
    "librispeech_asr",
    "other",
    split="validation",
    streaming=True,
    trust_remote_code=True
)

# Cast audio column
train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16000, decode=True))
val_dataset = val_dataset.cast_column("audio", Audio(sampling_rate=16000, decode=True))

print("✅ Dataset ready (streaming mode)")
print(f"Local storage used: 0 GB ✅\n")
print(f"🎯 Using LibriSpeech 'other' subset (more challenging than 'clean')")
print(f"   More diverse speakers, noisier audio → better generalization test")

# Inspect first sample
try:
    sample = next(iter(train_dataset))
    print("\nSample keys:", sample.keys())
    print(f"Audio shape: {sample['audio']['array'].shape}")
    print(f"Sample rate: {sample['audio']['sampling_rate']} Hz")
    print(f"Speaker ID: {sample.get('speaker_id', 'N/A')}")
except Exception as e:
    print(f"⚠️ Sample inspection failed: {e}")
    print("This is OK - audio will decode when training starts")

## Step 5: Load Enhanced Model (Same Architecture as LibriSpeech)

In [ ]:
import sys
import os

# Retrieve the voiceflow_ml_path from Step 3
try:
    %store -r voiceflow_ml_path
    print(f"✅ Using voiceflow-ml path: {voiceflow_ml_path}")
except:
    print("⚠️ Path not stored, searching...")
    possible_paths = [
        '/content/VoiceFlow-Intelligence-Platform/voiceflow-ml',
        '/content/VoiceFlow-Intelligence-Platform/VoiceFlow-Intelligence-Platform/voiceflow-ml',
    ]
    voiceflow_ml_path = None
    for path in possible_paths:
        if os.path.exists(os.path.join(path, 'models')):
            voiceflow_ml_path = path
            break
    
    if not voiceflow_ml_path:
        raise FileNotFoundError("Could not find voiceflow-ml directory. Please re-run Step 3.")

# Add to Python path
if voiceflow_ml_path not in sys.path:
    sys.path.insert(0, voiceflow_ml_path)

# Import model
from models.diarization.model import FastDiarizationModel
print("✅ Model imported successfully")

# Model configuration (SAME as LibriSpeech for fair comparison)
CONFIG = {
    'num_speakers': 2,
    'hidden_size': 256,
    'encoder_type': 'lightweight-cnn',
    'dropout': 0.3,
}

# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = FastDiarizationModel(**CONFIG).to(device)

print(f"\n{'='*60}")
print(f"Model: FastDiarizationModel (Enhanced CNN)")
print(f"Device: {device}")
print(f"Parameters: {model.count_parameters() / 1e6:.2f}M")
print(f"Trainable: {model.count_trainable_parameters() / 1e6:.2f}M")
print(f"{'='*60}\n")
print(f"🎯 Training on REAL diarization labels (AMI Corpus)")
print(f"   vs LibriSpeech pseudo-labels running in parallel")

## Step 6: Create Streaming DataLoader (LibriSpeech 'other')

In [ ]:
import torch
import torchaudio
from torch.utils.data import DataLoader, IterableDataset
import numpy as np

class StreamingAudioDataset(IterableDataset):
    """
    Wrapper for streaming HuggingFace dataset.
    Preprocesses audio on-the-fly (no storage).
    """
    
    def __init__(self, hf_dataset, target_sr=16000, duration=3.0, max_samples=None):
        self.dataset = hf_dataset
        self.target_sr = target_sr
        self.target_length = int(target_sr * duration)
        self.max_samples = max_samples
        
        # Track speaker IDs and assign balanced labels
        self.speaker_to_label = {}
        self.next_label = 0
    
    def __iter__(self):
        count = 0
        for sample in self.dataset:
            # Stop if max_samples reached
            if self.max_samples and count >= self.max_samples:
                break
            
            try:
                # Extract audio
                audio = torch.FloatTensor(sample['audio']['array'])
                sr = sample['audio']['sampling_rate']
                
                # Resample to 16kHz if needed
                if sr != self.target_sr:
                    resampler = torchaudio.transforms.Resample(sr, self.target_sr)
                    audio = resampler(audio)
                
                # Pad/crop to fixed length
                if audio.shape[0] > self.target_length:
                    # Random crop
                    start = np.random.randint(0, audio.shape[0] - self.target_length)
                    audio = audio[start:start + self.target_length]
                elif audio.shape[0] < self.target_length:
                    # Zero-pad
                    audio = torch.nn.functional.pad(audio, (0, self.target_length - audio.shape[0]))
                
                # Balanced label assignment (round-robin for 50/50 distribution)
                speaker_id = str(sample.get('speaker_id', 0))
                if speaker_id not in self.speaker_to_label:
                    self.speaker_to_label[speaker_id] = self.next_label
                    self.next_label = (self.next_label + 1) % CONFIG['num_speakers']
                
                label = self.speaker_to_label[speaker_id]
                
                yield audio, label
                count += 1
                
            except Exception as e:
                # Skip corrupted samples
                print(f"Skipping sample due to error: {e}")
                continue

# Create streaming datasets
train_streaming = StreamingAudioDataset(train_dataset, max_samples=10000)
val_streaming = StreamingAudioDataset(val_dataset, max_samples=1000)

# Create dataloaders
BATCH_SIZE = 32

def collate_fn(batch):
    audios, labels = zip(*batch)
    return torch.stack(audios), torch.LongTensor(labels)

train_loader = DataLoader(
    train_streaming,
    batch_size=BATCH_SIZE,
    collate_fn=collate_fn,
    num_workers=0
)

val_loader = DataLoader(
    val_streaming,
    batch_size=BATCH_SIZE,
    collate_fn=collate_fn,
    num_workers=0
)

print("✅ DataLoaders ready (streaming mode)")
print(f"   Training samples: 10,000 (same as LibriSpeech 'clean')")
print(f"   Validation samples: 1,000")
print(f"   Distribution: Balanced round-robin for {CONFIG['num_speakers']} classes")
print(f"\\n🎯 Testing on 'other' subset (more challenging, diverse audio)")
print(f"   Expected: Lower Val Acc than 'clean' (61.5%) due to noise")

## Step 7: Training Configuration (Same as LibriSpeech)

In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR

# Training hyperparameters (IDENTICAL to LibriSpeech for fair comparison)
TRAINING_CONFIG = {
    'num_epochs': 30,
    'learning_rate': 1e-3,
    'weight_decay': 1e-4,
    'warmup_epochs': 3,
    'save_every_n_epochs': 5,
    'validate_every_n_epochs': 1,
}

# Optimizer
optimizer = optim.AdamW(
    model.parameters(),
    lr=TRAINING_CONFIG['learning_rate'],
    weight_decay=TRAINING_CONFIG['weight_decay']
)

# Learning rate scheduler
scheduler = CosineAnnealingLR(
    optimizer,
    T_max=TRAINING_CONFIG['num_epochs'],
    eta_min=1e-6
)

# Loss function
criterion = torch.nn.CrossEntropyLoss()

print("✅ Training configuration ready")
print(f"\nHyperparameters (same as LibriSpeech):")
for key, value in TRAINING_CONFIG.items():
    print(f"  {key}: {value}")

## Step 8: Checkpoint Resume Logic

In [ ]:
import glob

def load_latest_checkpoint(checkpoint_dir, model, optimizer, scheduler=None):
    """
    Load the latest checkpoint if available.
    Returns: start_epoch, best_val_acc
    """
    checkpoints = glob.glob(f"{checkpoint_dir}/checkpoint_epoch*.pth")
    
    if not checkpoints:
        print("No checkpoint found. Starting from scratch.")
        return 0, 0.0
    
    # Find latest checkpoint
    latest_checkpoint = max(checkpoints, key=os.path.getctime)
    print(f"Loading checkpoint: {latest_checkpoint}")
    
    checkpoint = torch.load(latest_checkpoint, map_location=device)
    
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    if scheduler and 'scheduler_state_dict' in checkpoint:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    
    start_epoch = checkpoint['epoch'] + 1
    best_val_acc = checkpoint.get('best_val_acc', 0.0)
    
    print(f"✅ Resumed from epoch {checkpoint['epoch']}")
    print(f"   Best validation accuracy: {best_val_acc:.2f}%\n")
    
    return start_epoch, best_val_acc

# Load checkpoint if exists
start_epoch, best_val_acc = load_latest_checkpoint(
    checkpoint_dir, model, optimizer, scheduler
)

## Step 9: Training Loop

In [ ]:
from tqdm.notebook import tqdm
import time

def train_epoch(model, train_loader, criterion, optimizer, device, epoch):
    """
    Train for one epoch.
    """
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    num_batches = 0
    
    pbar = tqdm(train_loader, desc=f"[LibriSpeech-OTHER] Epoch {epoch+1}/{TRAINING_CONFIG['num_epochs']}")
    
    for batch_idx, (audio, labels) in enumerate(pbar):
        # Move to GPU
        audio = audio.to(device)
        labels = labels.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(audio)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Metrics
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        num_batches += 1
        
        # Update progress bar
        pbar.set_postfix({
            'loss': total_loss / num_batches,
            'acc': 100.0 * correct / total
        })
    
    avg_loss = total_loss / num_batches
    accuracy = 100.0 * correct / total
    
    return avg_loss, accuracy


def validate(model, val_loader, criterion, device):
    """
    Validate model.
    """
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    num_batches = 0
    
    with torch.no_grad():
        for audio, labels in tqdm(val_loader, desc="[LibriSpeech-OTHER] Validation"):
            audio = audio.to(device)
            labels = labels.to(device)
            
            outputs = model(audio)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            num_batches += 1
    
    avg_loss = total_loss / num_batches
    accuracy = 100.0 * correct / total
    
    return avg_loss, accuracy


# Training loop
print("\n" + "="*60)
print("🎯 Starting Training on LibriSpeech 'other' (Challenging Audio)")
print("="*60 + "\n")

training_start = time.time()

for epoch in range(start_epoch, TRAINING_CONFIG['num_epochs']):
    epoch_start = time.time()
    
    # Train
    train_loss, train_acc = train_epoch(
        model, train_loader, criterion, optimizer, device, epoch
    )
    
    # Validate
    val_loss, val_acc = None, None
    if (epoch + 1) % TRAINING_CONFIG['validate_every_n_epochs'] == 0:
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        
        # Update best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_path = f"{checkpoint_dir}/best_model.pth"
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'val_acc': val_acc,
            }, best_model_path)
            print(f"✅ New best model saved! Val Acc: {val_acc:.2f}%")
    
    # Update learning rate
    scheduler.step()
    
    # Epoch summary
    epoch_time = time.time() - epoch_start
    print(f"\n[LibriSpeech-OTHER] Epoch {epoch+1}/{TRAINING_CONFIG['num_epochs']} Summary:")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    if val_acc is not None:
        print(f"  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.2f}%")
        print(f"  🎯 Compare to LibriSpeech 'clean': Val Acc ~61.5%")
    print(f"  Time: {epoch_time:.1f}s | LR: {scheduler.get_last_lr()[0]:.2e}\n")
    
    # Save checkpoint
    if (epoch + 1) % TRAINING_CONFIG['save_every_n_epochs'] == 0:
        checkpoint_path = f"{checkpoint_dir}/checkpoint_epoch{epoch+1}.pth"
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'train_loss': train_loss,
            'val_acc': val_acc if val_acc else 0.0,
            'best_val_acc': best_val_acc,
        }, checkpoint_path)
        print(f"💾 Checkpoint saved: {checkpoint_path}\n")

training_time = time.time() - training_start
print("\n" + "="*60)
print("🎉 LibriSpeech 'other' Training Complete!")
print("="*60)
print(f"Total time: {training_time/3600:.2f} hours")
print(f"Best validation accuracy: {best_val_acc:.2f}%")
print(f"Best model saved at: {checkpoint_dir}/best_model.pth")
print(f"\n🎯 Compare to LibriSpeech 'clean': ~61.5% Val Acc")
print(f"   Difference: {best_val_acc - 61.5:.1f}% ({'harder' if best_val_acc < 61.5 else 'easier'} than 'clean')")

## Step 10: Export Trained Model to ONNX

In [ ]:
# Load best model
best_model_path = f"{checkpoint_dir}/best_model.pth"
checkpoint = torch.load(best_model_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"Loaded best LibriSpeech 'other' model (Val Acc: {checkpoint['val_acc']:.2f}%)\n")

# Export to ONNX
dummy_input = torch.randn(1, 48000).to(device)  # 3 seconds @ 16kHz
onnx_path = f"{checkpoint_dir}/fast_cnn_librispeech_other_trained.onnx"

torch.onnx.export(
    model,
    dummy_input,
    onnx_path,
    input_names=['audio'],
    output_names=['speaker_logits'],
    dynamic_axes={
        'audio': {0: 'batch_size'},
        'speaker_logits': {0: 'batch_size'}
    },
    opset_version=14
)

print(f"✅ ONNX model exported: {onnx_path}")
print(f"Model size: {os.path.getsize(onnx_path) / (1024**2):.2f} MB")

# Download ONNX model
from google.colab import files
files.download(onnx_path)

## 🎉 LibriSpeech 'other' Training Complete!

**Comparison with LibriSpeech 'clean'**:

| Metric | LibriSpeech 'clean' | LibriSpeech 'other' | Difference |
|--------|---------------------|---------------------|------------|
| Val Acc | 61.50% | TBD | TBD |
| Train Acc | ~48% | TBD | TBD |
| Variance | ±15% | TBD | TBD |
| Audio Quality | Studio clean | Noisier | More challenging |

**Next Steps**:
1. Compare ONNX models from both trainings
2. Benchmark inference speed (should be identical)
3. Choose best model (likely 'clean' for production)
4. Document findings in PARALLEL_TRAINING_STRATEGY.md

**Key Insight**: This experiment tests model robustness on noisier audio vs clean studio recordings!